In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
#import os
from utils.model3 import Net_mask
#from utils.data import Data
#from utils.loss_experiment import Loss
#from scripts.parameters import get_parameters
import torch
#import numpy as np
import matplotlib.pyplot as plt
from torch.distributions.multivariate_normal import MultivariateNormal

In [3]:
model_ML_inf = Net_mask()
model_MAP_e4 = Net_mask()
model_MAP_e5 = Net_mask()
model_MAP_e6 = Net_mask()
model_ML_inf_reg = Net_mask()
model_ML_inf.load_state_dict(torch.load(f'../saved_models/model_infinite/2/model_weights.pth'))
model_MAP_e4.load_state_dict(torch.load(f'../saved_models/finite_long_e4/map/0/model_weights.pth'))
model_MAP_e5.load_state_dict(torch.load(f'../saved_models/finite_long_e5/map/0/model_weights.pth'))
model_MAP_e6.load_state_dict(torch.load(f'../saved_models/finite_long_e6/map/0/model_weights.pth'))
model_ML_inf_reg.load_state_dict(torch.load(f'../saved_models/model_infinite/3_regularization/model_weights.pth'))

<All keys matched successfully>

In [4]:
tensor_D = torch.tensor(
    [[1, -1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, -1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0,  1, -1, 0, 0, 0, 0, 0, 0],
    [0, 0,  0, 1, -1, 0, 0, 0, 0, 0],
    [0, 0,  0, 0, 1, -1, 0, 0, 0, 0],
    [0, 0,  0, 0, 0, 1, -1, 0, 0, 0],
    [0, 0,  0, 0, 0, 0, 1, -1, 0, 0],
    [0, 0,  0, 0, 0, 0, 0, 1, -1, 0],
    [0, 0,  0, 0, 0, 0, 0, 0, 1, -1],
    [-1, 0, 0, 0, 0, 0, 0, 0, 0, 1],], dtype=torch.float
)
tensor_Q_m  = torch.mm(torch.t(tensor_D), tensor_D)
n_param = tensor_D.size(dim=0)
#tau2 = 1000 # 1/tau2 is the noise added to the diagonal
#tensor_Q_m_modified = tensor_Q_m + torch.eye(n_param)*(1./tau2)
#sigma2_eps = 0.01  # 1/sigma2_eps is the factor before the likelihood
#tensor_mu_m = torch.zeros(n_param)

#tensor_Sigma_m = torch.inverse(tensor_Q_m_modified)
#tensor_Sigma_eps = torch.eye(n_param)*sigma2_eps
#tensor_mu_eps = tensor_mu_m 

In [5]:
all_test_obses = torch.load(f'../data/test_observations/test_set_full/all_test_obses.pt')
all_test_solutions = torch.load(f'../data/test_observations/test_set_full/all_test_solutions.pt')

In [6]:
def loss(predicted, analytical):
    difference = predicted - analytical
    squared_error = torch.square(difference)
    tensor_squared_error_sum = torch.sum(squared_error, dim=1)
    tensor_squared_error_sum_all = torch.sum(tensor_squared_error_sum)
    return tensor_squared_error_sum_all

In [65]:
max_points = 6
n_bnn = 100
n_data = all_test_obses.shape[1]

loss_ML_inf = torch.zeros(max_points)
loss_ML_inf_reg = torch.zeros(max_points)
loss_MAP_e4 = torch.zeros(max_points)
loss_MAP_e5 = torch.zeros(max_points)
loss_MAP_e6 = torch.zeros(max_points)

loss_RML_e4_0 = torch.zeros(max_points)
loss_RML_e4_mean = torch.zeros(max_points)
loss_RML_e4_median = torch.zeros(max_points)
loss_RML_e4_variance = torch.zeros(max_points)
loss_RML_e5_0 = torch.zeros(max_points)
loss_RML_e5_mean = torch.zeros(max_points)
loss_RML_e5_median = torch.zeros(max_points)
loss_RML_e5_variance = torch.zeros(max_points)

for p in range(max_points):
    test_set_points = all_test_obses[p]
    test_set_solution = all_test_solutions[p]

    pred_RML_e4_all = torch.zeros(n_bnn, n_data, n_param)
    pred_RML_e5_all = torch.zeros(n_bnn, n_data, n_param)
    for i in range(n_bnn):
        model_RML_e5 = Net_mask()
        model_RML_e5.load_state_dict(torch.load(f'../saved_models/rml_100/pos/{i}/model_weights.pth'))
        pred_RML_e5_all[i] = model_RML_e5(test_set_points)

        model_RML_e4 = Net_mask()
        model_RML_e4.load_state_dict(torch.load(f'../saved_models/rml_100_1.0e+04/pos/{i}/model_weights.pth'))
        pred_RML_e4_all[i] = model_RML_e5(test_set_points)

    loss_ML_inf[p] = loss(model_ML_inf(test_set_points), test_set_solution)
    loss_ML_inf_reg[p] = loss(model_ML_inf_reg(test_set_points), test_set_solution)
    loss_MAP_e4[p] = loss(model_MAP_e4(test_set_points), test_set_solution)
    loss_MAP_e5[p] = loss(model_MAP_e5(test_set_points), test_set_solution)
    loss_MAP_e6[p] = loss(model_MAP_e6(test_set_points), test_set_solution)

    loss_RML_e4_0[p] = loss(pred_RML_e4_all[0], test_set_solution)
    loss_RML_e4_mean[p] = loss(torch.mean(pred_RML_e4_all, dim=0), test_set_solution)
    loss_RML_e4_median[p] = loss(torch.median(pred_RML_e4_all, dim=0)[0], test_set_solution)
    loss_RML_e4_variance[p] = torch.mean(torch.var(pred_RML_e4_all, dim=0))

    loss_RML_e5_0[p] = loss(pred_RML_e5_all[0], test_set_solution)
    loss_RML_e5_mean[p] = loss(torch.mean(pred_RML_e5_all, dim=0), test_set_solution)
    loss_RML_e5_median[p] = loss(torch.median(pred_RML_e5_all, dim=0)[0], test_set_solution)
    loss_RML_e5_variance[p] = torch.mean(torch.var(pred_RML_e5_all, dim=0))

In [64]:
a = torch.reshape(torch.range(1, 24, 1), (2, 3, 4))
print(a)
#print(torch.mean(a, dim=0).shape)
#print(torch.median(a, dim=0)[0].shape)
#print(torch.mean(torch.var(a, dim=2), dim=1))
print(torch.mean(torch.var(a, dim=0)))
#print(a[:,0,0])


tensor([[[ 1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.],
         [ 9., 10., 11., 12.]],

        [[13., 14., 15., 16.],
         [17., 18., 19., 20.],
         [21., 22., 23., 24.]]])
tensor(72.)


/var/folders/pq/v4rj59917053v7zk70nqchsc0000gn/T/ipykernel_63414/2994698491.py:1: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  a = torch.reshape(torch.range(1, 24, 1), (2, 3, 4))


In [92]:
title = f'{"Type":13} | {"data":4} | {"1 point":7} | {"2 points":7} | {3:7} | {4:7} | {5:7} | {6:7} |'

string_ML_inf = f'{"ML-inf":13} | {"inf":4} |'
string_ML_inf_reg = f'{"ML-inf-reg":13} | {"inf":4} |'

string_MAP_e4 = f'{"MAP-e4":13} | {"1e4":4} |'
string_MAP_e5 = f'{"MAP-e5":13} | {"1e5":4} |'
string_MAP_e6 = f'{"MAP-e6":13} | {"1e6":4} |'

string_RML_e4_0 = f'{"RML-e4-0":13} | {"1e4":4} |'
string_RML_e4_mean = f'{"RML-e4-mean":13} | {"1e4":4} |'
string_RML_e4_median = f'{"RML-e4-median":13} | {"1e4":4} |'

string_RML_e5_0 = f'{"RML-e5-0":13} | {"1e5":4} |'
string_RML_e5_mean = f'{"RML-e5-mean":13} | {"1e5":4} |'
string_RML_e5_median = f'{"RML-e5-median":13} | {"1e5":4} |'

string_RML_e6_0 = f'{"RML-e6-0":13} | {"1e6":4} |'
string_RML_e6_mean = f'{"RML-e6-mean":13} | {"1e6":4} |'
string_RML_e6_median = f'{"RML-e6-median":13} | {"1e6":4} |'

string_RML_e4_var = f'{"RML-e4-var":13} | {"1e4":4} |'
string_RML_e5_var = f'{"RML-e5-var":13} | {"1e5":4} |'


for i in range(max_points):

    string_ML_inf += f' {loss_ML_inf[i]:7.2f} |' 
    string_ML_inf_reg += f' {loss_ML_inf_reg[i]:7.2f} |' 

    string_MAP_e4 += f' {loss_MAP_e4[i]:7.2f} |' 
    string_MAP_e5 += f' {loss_MAP_e5[i]:7.2f} |' 
    string_MAP_e6 += f' {loss_MAP_e6[i]:7.2f} |' 

    string_RML_e4_0 += f' {loss_RML_e4_0[i]:7.2f} |' 
    string_RML_e4_mean += f' {loss_RML_e4_mean[i]:7.2f} |' 
    string_RML_e4_median += f' {loss_RML_e4_median[i]:7.2f} |' 

    string_RML_e5_0 += f' {loss_RML_e5_0[i]:7.2f} |' 
    string_RML_e5_mean += f' {loss_RML_e5_mean[i]:7.2f} |' 
    string_RML_e5_median += f' {loss_RML_e5_median[i]:7.2f} |' 

    string_RML_e4_var += f' {loss_RML_e4_variance[i]:7.4f} |' 
    string_RML_e5_var += f' {loss_RML_e5_variance[i]:7.4f} |' 


string_break = '-'*82
total_string = ""

print(title); total_string+=title+'\n'
print(string_break); total_string+=string_break+'\n'
print(string_ML_inf); total_string+=string_ML_inf+'\n'
print(string_ML_inf_reg); total_string+=string_ML_inf_reg+'\n'
print(string_break); total_string+=string_break+'\n'

print(string_MAP_e4); total_string+=string_MAP_e4+'\n'
print(string_MAP_e5); total_string+=string_MAP_e5+'\n'
print(string_MAP_e6); total_string+=string_MAP_e6+'\n'
print(string_break); total_string+=string_break+'\n'

print(string_RML_e4_0); total_string+=string_RML_e4_0+'\n'
print(string_RML_e4_mean); total_string+=string_RML_e4_mean+'\n'
print(string_RML_e4_median); total_string+=string_RML_e5_median+'\n'
print(string_break); total_string+=string_break+'\n'

print(string_RML_e5_0); total_string+=string_RML_e5_0+'\n'
print(string_RML_e5_mean); total_string+=string_RML_e5_mean+'\n'
print(string_RML_e5_median); total_string+=string_RML_e5_median+'\n'
print(string_break); total_string+=string_break+'\n'

print(string_RML_e4_var); total_string+=string_RML_e4_var+'\n'
print(string_RML_e5_var); total_string+=string_RML_e5_var+'\n'
print(string_break); total_string+=string_break+'\n'


Type          | data |       1 |       2 |       3 |       4 |       5 |       6 |
----------------------------------------------------------------------------------
ML-inf        | inf  |  604.61 |   52.35 |  184.63 |  137.34 |   80.66 |   38.61 |
ML-inf-reg    | inf  |  383.72 |   35.46 |  146.53 |   99.94 |   50.30 |   25.35 |
----------------------------------------------------------------------------------
MAP-e4        | 1e4  | 3959.59 |  832.50 |  499.79 |  315.37 |  183.56 |  105.37 |
MAP-e5        | 1e5  |  606.07 |   81.62 |  210.48 |  144.25 |   77.47 |   41.24 |
MAP-e6        | 1e6  |  650.88 |   73.00 |  214.08 |  145.07 |   81.68 |   40.78 |
----------------------------------------------------------------------------------
RML-e4-0      | 1e4  | 5614.71 | 1583.37 |  950.27 |  589.90 |  368.71 |  219.93 |
RML-e4-mean   | 1e4  | 5536.79 | 1534.91 |  893.95 |  542.87 |  323.31 |  186.98 |
RML-e4-median | 1e4  | 5549.08 | 1537.36 |  895.25 |  544.19 |  325.42 |  189.25 |
----

In [93]:
#path_to_output_folder = '../figures/plot_table_prediction_error/output.txt'
#path_to_output_folder = '../saved_models/rml_100_1.0e+04/' + run_type + '/'
#if os.path.exists(path_to_output_folder) and os.path.isdir(path_to_output_folder):
#    shutil.rmtree(path_to_output_folder)
#os.makedirs(path_to_output_folder)
with open('../figures/plot_table_prediction_error/plot_table_prediction_error.txt', 'w') as outfile:
    outfile.write(total_string[:-1])